In [16]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

#Neural Network from Scratch

In [17]:
class Inicio(): #Define as propriedades de uma camada única, caso deseja-se mais de uma é só ir alterando os valores de inputs
  def __init__(self):
    pass

  def pesos_biases(self, n_inputs, n_neuronios): #Define a o num. de dados do input e o num. de neurons da camada oculta

    pesos_camada_hidden = np.random.randn(n_neuronios, n_inputs.shape[0]) #Gera uma matriz de dimensão n_inputs (linhas) n_neurons (colunas). Composta por valores aleatórios normalizados
    biases_camada_hidden = np.zeros((1, n_neuronios)) #Inicia uma matriz de 0's de dimensão 1 (linha) e n_neurons (colunas)

    pesos_camada_output = np.random.randn(n_neuronios, n_inputs.shape[0]) #Gera uma matriz de dimensão n_inputs (linhas) n_neurons (colunas). Composta por valores aleatórios normalizados
    biases_camada_output = np.zeros((1, n_neuronios)) #Inicia uma matriz de 0's de dimensão 1 (linha) e n_neurons (colunas)

    dic_pesos_biases = {"pesos_camada_hidden": pesos_camada_hidden, "biases_camada_hidden": biases_camada_hidden, "pesos_camada_output": pesos_camada_output, "biases_camada_output": biases_camada_output}
    return dic_pesos_biases

In [18]:
class Forward():
  def __init__(self):
    pass

  def ativacao_sigmoide(self, saida): #Define a função de ativação dos output
      return 1/(1+np.exp(-saida))

  def propagacao_direta(self, inputs, dic_pesos_biases): #Realiza a propagacao direta dos dados do input
    saida1 = np.dot(inputs, dic_pesos_biases['pesos_camada_hidden']) + dic_pesos_biases['biases_camada_hidden'] #input vezes peso + bias
    saida1_ativada = self.ativacao_sigmoide(saida1)

    saida2 = np.dot(saida1, dic_pesos_biases['pesos_camada_output']) + dic_pesos_biases['biases_camada_output'] #input vezes peso + bias
    saida2_ativada = self.ativacao_sigmoide(saida2)

    saidas = {'saida1': saida1 ,"saida2": saida2 , "saida1_ativada": saida1_ativada , "saida2_ativada": saida2_ativada}
    return saida2_ativada, saidas

In [19]:
class Perda():
  def calculate(self, output_active, y): #Recebe os outputs rede e os valores verdadeiros (y) como entrada
    perdas_amostras_individuais = self.forward(output_active, y) #Calcula as perdas individuais para cada exemplo
    data_loss = np.mean(perdas_amostras_individuais) #Calcula a média das perdas individuais
    return data_loss

class Loss_CategoricalCrossentropy(Perda):
  def forward(self, y_pred, y_true): #Recebe as previsões (y_pred) e os valores verdadeiros (y_true)
    samples = len(y_pred)
    y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7) #Garantir que os valores previstos estejam dentro de um intervalo específico

    if len(y_true.shape) == 1: #Apenas os índices das classes
      confiancas_corretas = y_pred_clipped[range(samples), y_true]
    elif len(y_true.shape) == 2: #Para situações em que os valores sofreram o One-Hot Encoder
      confiancas_corretas = np.sum(y_pred_clipped*y_true, axis=1)

    perda_amostras_individuais = -np.log(confiancas_corretas) #Calcula a perda para cada exemplo individual

    return perda_amostras_individuais

In [20]:
class Backward():
  def __init__(self):
    pass

  def backpropagation(self, saidas, dic_pesos_biases, y_true):
    n_neuronios = 29

    loss_calculator = Loss_CategoricalCrossentropy()  # Instanciar a classe de cálculo de perda
    perdas_amostras_individuais = loss_calculator.forward(saidas['saida2_ativada'], y_true)  # Calcular as perdas individuais da ultima camada
    diferenca = perdas_amostras_individuais

#Para os pesos multiplicamos esse "Diferença mínima" pela primeira camada transposta (aplicada na função de ativação) e dividimos pela quantidade de neurônios
    novos_pesos_camada_op = (diferenca*saidas['saida2_ativada'])/n_neuronios
#Para os bias tiramos a média dessa "Diferença mz2_activeínima"
    novos_biases_camada_op = (np.sum(diferenca, keepdims=True)) #Vale ressaltar que o "keepdims" mantém em formato de array

    derivada_c1 = np.multiply(np.dot(dic_pesos_biases["pesos_camada_output"].T, novos_pesos_camada_op), 1 - np.power(saidas["saida1_ativada"], 2))
    derivada_peso1 = (1/n_neuronios)*np.dot(derivada_c1, x.T)
    derivada_biases1 = (1/n_neuronios)*np.sum(derivada_c1, keepdims=True)

    gradientes = {"dp1": derivada_peso1, "db1": derivada_biases1, "dp2": novos_pesos_camada_op, "db2": novos_biases_camada_op}

    return gradientes

In [21]:
class Gradiente():
  def __init__(self):
      pass

  def descente(self, gradientes, dic_pesos_biases, learning_rate=0.01):
    p1 = dic_pesos_biases['pesos_camada_hidden']
    b1 = dic_pesos_biases['bias_camada_hidden']
    p2 = dic_pesos_biases['pesos_camada_output']
    b2 = dic_pesos_biases['bias_camada_output']

    dp1 = gradientes['dp1']
    db1 = gradientes['db1']
    dp2 = gradientes['dp2']
    db2 = gradientes['db2']

    w1 = p1 - learning_rate * dp1
    b1 = b1 - learning_rate * db1
    w2 = p2 - learning_rate * dp2
    b2 = b2 - learning_rate * db2

    dic_pesos_biases = {"pesos_camada_hidden": w1, "bias_camada_hidden": b1, "pesos_camada_output": w2, "bias_camada_output": b2}

    return dic_pesos_biases

In [22]:
class Perceptron():
  def __init__(self):
      pass

  def fit(self, x_treinamento, y_treinamento, epocas):
    n_features = x_treinamento.shape[1]
    inicializacao = Inicio()
    perda = Perda()
    self.dic_pesos_biases = inicializacao.pesos_biases(x_treinamento, n_features)

    pesos_hidden = self.dic_pesos_biases['pesos_camada_hidden']
    biases_hidden = self.dic_pesos_biases['biases_camada_hidden']
    pesos_output = self.dic_pesos_biases['pesos_camada_output']
    biases_output = self.dic_pesos_biases['biases_camada_output']

    forward = Forward()
    backward = Backward()
    gradiente = Gradiente()

    for i in range(0, epocas):
      saida2_ativada, saidas = forward.propagacao_direta(x_treinamento, self.dic_pesos_biases)
      total_error = perda.calculate()
      if((i % 10 == 0)or(i == 1)):
        print(f"Epoch {i} - Total error: {total_error} \n Accuracy: {((1-total_error)*100):.2f}%")

      gradientes = backward.backpropagation(saidas, self.dic_pesos_biases, y_true)
      self.dic_pesos_biases = gradiente.descente(gradientes, self.dic_pesos_biases, learning_rate=0.01)

    return self.dic_pesos_biases

  def prediction(self, x_teste):
    forward = Forward()
    saida2_ativada, saidas = forward.propagation(x_teste, self.dic_pesos_biases)
    predictions = np.round(saidas["saida2_ativada"])

    return predictions

#Análise Exploratória

In [23]:
#Importando a base de dados que será utilizada
creditCard = pd.read_csv("/content/drive/MyDrive/CIS_3Periodo/creditcard.csv")
creditCard.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [24]:
#Verificando se há elementos vazios
creditCard.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [25]:
#Realiza uma contagem de quantos itens cada rótulo possui
creditCard["Class"].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [26]:
fraudes = creditCard[creditCard.Class == 1]
normal = creditCard[creditCard.Class == 0]

In [27]:
#Definindo que meu dataset contenha todas as fraudulentas e 1000 não fraudulentas
newCreditCard = fraudes.sample(frac=0.8)

#Definindo como 80% das transações normais
newCreditCard = pd.concat([newCreditCard, normal.loc[0:1000]], axis = 0)
newCreditCard = shuffle(newCreditCard)

In [28]:
newCreditCard["Class"].value_counts()

0    999
1    394
Name: Class, dtype: int64

In [38]:
x = np.array(newCreditCard.drop(['Class'], axis = 1))
y = np.array(newCreditCard["Class"])

In [39]:
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, test_size=0.2, random_state=0)

Peço desculpas, devido a alguns problemas familiares (minha vó, que mora comigo, está no hospital), não consegui finalizar o desenvolvimento da atividade.